In [ ]:
%%capture
!uv pip install torchcodec torch
!uv pip install datasets[audio]
!uv pip install evaluate
!uv pip install huggingface_hub
!uv pip install ipywidgets
!uv pip install git+https://github.com/huggingface/transformers.git
!uv pip install jiwer
!uv pip install accelerate

Resolved 26 packages in 55ms                                         
⠙ Preparing packages... (0/25)                                                  
⠙ Preparing packages... (0/25)-----------------     0 B/932.39 KiB      
⠙ Preparing packages... (0/25)----------------- 14.93 KiB/932.39 KiB    
⠙ Preparing packages... (0/25)----------------- 14.93 KiB/932.39 KiB    
jinja2                   ------------------------------     0 B/131.74 KiB
⠙ Preparing packages... (0/25)----------------- 14.93 KiB/932.39 KiB    
jinja2                   ------------------------------     0 B/131.74 KiB
⠙ Preparing packages... (0/25)----------------- 14.93 KiB/932.39 KiB    
jinja2                   ------------------------------     0 B/131.74 KiB
⠙ Preparing packages... (0/25)----------------- 14.93 KiB/932.39 KiB    
nvidia-nvtx-cu12         ------------------------------     0 B/87.85 KiB
jinja2                   ------------------------------     0 B/131.74 KiB
⠙ Preparing packages... (0/25)-------

In [22]:
!uv pip install huggingface_hub
!uv pip install ipywidgets
!uv pip install git+https://github.com/huggingface/transformers.git


Using Python 3.11.13 environment at: /teamspace/studios/this_studio/.venv
Audited 1 package in 4ms
Using Python 3.11.13 environment at: /teamspace/studios/this_studio/.venv
Audited 1 package in 8ms
Using Python 3.11.13 environment at: /teamspace/studios/this_studio/.venv
Resolved 25 packages in 136ms                                        
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)--------------     0 B/3.10 MiB            
⠙ Preparing packages... (0/3)-------------- 14.94 KiB/3.10 MiB          
⠙ Preparing packages... (0/3)-------------- 14.94 KiB/3.10 MiB          
regex                ------------------------------     0 B/781.55 KiB
⠙ Preparing packages... (0/3)-------------- 14.94 KiB/3.10 MiB          
regex                ------------------------------     0 B/781.55 KiB
⠙ Preparing packages... (0/3)-------------- 14.94 KiB/3.10 MiB          
regex                ------------------------------     0 B/781.55 KiB

In [1]:
from huggingface_hub import notebook_login

# 
notebook_login()


In [2]:
import os

os.chdir('./mms')

## Pre-processing

## Dataset

In [4]:
#del dataset_global
#del dataset
#del dataset_val

In [4]:
from datasets import load_dataset, Audio

dataset_global = load_dataset("csv", data_files="dataset.csv", split="train")
dataset_global = dataset_global.remove_columns(["client_id", "sentence_id", "sentence_domain", "up_votes", "down_votes", "age", "gender", "accents", "variant", "locale", "segment"])
dataset_global = dataset_global.cast_column("audio", Audio(sampling_rate=16000))
print(dataset_global[0]["audio"])

dataset = dataset_global.train_test_split(test_size=0.2)
dataset_val = dataset["train"].train_test_split(test_size=0.2)
dataset["train"] = dataset_val["train"]
dataset["validation"] = dataset_val["test"]


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

### Vocabulary

In [ ]:
import re

PUNCT = re.compile(r'[\፡\።\፣\፤\፥\፦\፧\፨\፠]')
def remove_punctuation(sample):
  sample["sentence"] = PUNCT.sub('', sample["sentence"]).lower()
  return sample

dataset["train"] = dataset["train"].map(remove_punctuation)
dataset["test"] = dataset["test"].map(remove_punctuation)
dataset["validation"] = dataset["validation"].map(remove_punctuation)

dataset["train"].deco

Map: 100%|██████████| 257/257 [00:00<00:00, 11191.55 examples/s]


In [15]:
show_random_elements(dataset["train"].remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,የማንኛውም የውጭ ጣልቃ ገብነት ዓላማም ሰላምን እና ድርድርን በተመለከተ ብቻ ሊሆን ይገባል
1,የቡዳፔስቷ ፓውላ አርቴሚስ የወር አበባ ህመም ማስታገሻ ልብስ ገበያ ላይ የሚቀርብበትን ወቅት በጉጉት አየጠበቀች ትገኛለች
2,ለዚህ ያበቁህ ገዳዮችህ ተጠያቂ ሆነዋል እላለሁ
3,ኢትዮጵያ ይህ ስምምነት ጥቅሜን ይጎዳል በማለት በመቃወሟ ድርድሩ ሳይቋጭ ቀርቷል
4,እርሱን የማታልልበት ሌላ ዘዴ አለኝ አለች
5,ቢቢሲ በበኩሉ የሮያል ካናዳ ፖሊሰን እና ፒርሰን አውሮፕላን ማረፊያ ለማነጋገር ያደረው ጥረት አልተሳካም
6,የዩኬ ዜጎች በአውሮፕላን ዜጎችን ለምን ማውጣት አልተቻለም ሲሉ ቅሬታ አሰምተዋል
7,ሽማግሌዎቹም ሠላም ለማውረድ ተወያዩ
8,ግራ ዳልዬ አካባቢ ችግር ገጥሞኝ መቆም ግድ ሆኖብኝ ነበር
9,በኦሮሚያ እና በሌሎች አካባቢዎች የተዳከሙ አብያተ ክርስቲያናትን ቋንቋውን በሚያውቁ አገልጋዮች እና በአገልግሎት ማጠናከር


In [16]:
# Make of vocabulary
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = dataset["train"].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset["train"].column_names)
vocab_test = dataset["test"].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset["test"].column_names)
vocab_validation = dataset["validation"].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset["validation"].column_names)

Map: 100%|██████████| 257/257 [00:00<00:00, 75878.93 examples/s]


In [18]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

vocab_dict

{'!': 1,
 '-': 2,
 ':': 3,
 '?': 4,
 'ሀ': 5,
 'ሁ': 6,
 'ሂ': 7,
 'ሃ': 8,
 'ሄ': 9,
 'ህ': 10,
 'ሆ': 11,
 'ለ': 12,
 'ሉ': 13,
 'ሊ': 14,
 'ላ': 15,
 'ሌ': 16,
 'ል': 17,
 'ሎ': 18,
 'ሏ': 19,
 'ሐ': 20,
 'ሑ': 21,
 'ሓ': 22,
 'ሔ': 23,
 'ሕ': 24,
 'መ': 25,
 'ሙ': 26,
 'ሚ': 27,
 'ማ': 28,
 'ሜ': 29,
 'ም': 30,
 'ሞ': 31,
 'ሟ': 32,
 'ሠ': 33,
 'ሡ': 34,
 'ሣ': 35,
 'ሥ': 36,
 'ሦ': 37,
 'ረ': 38,
 'ሩ': 39,
 'ሪ': 40,
 'ራ': 41,
 'ሬ': 42,
 'ር': 43,
 'ሮ': 44,
 'ሯ': 45,
 'ሰ': 46,
 'ሱ': 47,
 'ሲ': 48,
 'ሳ': 49,
 'ሴ': 50,
 'ስ': 51,
 'ሶ': 52,
 'ሷ': 53,
 'ሸ': 54,
 'ሹ': 55,
 'ሺ': 56,
 'ሻ': 57,
 'ሼ': 58,
 'ሽ': 59,
 'ሾ': 60,
 'ቀ': 61,
 'ቁ': 62,
 'ቂ': 63,
 'ቃ': 64,
 'ቄ': 65,
 'ቅ': 66,
 'ቆ': 67,
 'ቋ': 68,
 'በ': 69,
 'ቡ': 70,
 'ቢ': 71,
 'ባ': 72,
 'ቤ': 73,
 'ብ': 74,
 'ቦ': 75,
 'ቧ': 76,
 'ቨ': 77,
 'ቪ': 78,
 'ቫ': 79,
 'ቬ': 80,
 'ቭ': 81,
 'ቮ': 82,
 'ተ': 83,
 'ቱ': 84,
 'ቲ': 85,
 'ታ': 86,
 'ቴ': 87,
 'ት': 88,
 'ቶ': 89,
 'ቷ': 90,
 'ቸ': 91,
 'ቹ': 92,
 'ቺ': 93,
 'ቻ': 94,
 'ቼ': 95,
 'ች': 96,
 'ቾ': 97,
 'ቿ': 98,
 'ኀ': 99,
 'ኃ': 100,
 'ኅ': 10

In [19]:
target_lang = "amh"
new_vocab_dict = {target_lang: vocab_dict}

In [20]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(new_vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", target_lang=target_lang)


In [14]:
#repo_name = "wav2vec2-large-mms-1b-amharique-colab"
#tokenizer.push_to_hub(repo_name)

## Feature extractor

A Wav2Vec2FeatureExtractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal 22.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be zero-mean-unit-variance normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an attention_mask for batched inference. In general, XLS-R models checkpoints should always use the attention_mask.


In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [29]:
dataset["train"][0]["audio"]